In [47]:
%matplotlib inline
import cv2
import mediapipe as mp
import matplotlib.pyplot as plt
import numpy as np
from skimage import data, img_as_float
from skimage import exposure


import csv
import os
import matplotlib.pyplot as plt

import tensorflow as tf

In [48]:
import itertools

def pre_process_landmark(landmark_list):

    # Convert to relative coordinates
    base_x, base_y = 0, 0
    for index, landmark_point in enumerate(landmark_list):
        if index == 0:
            base_x, base_y = landmark_point[0], landmark_point[1]

        landmark_list[index][0] = np.abs(landmark_list[index][0] - base_x)
        landmark_list[index][1] = np.abs(landmark_list[index][1] - base_y)

    flattened = []
    for i in range(landmark_list.shape[0]):
        flattened.append(landmark_list[i][0])
        flattened.append(landmark_list[i][1])


    normalized = []
    # Normalization
    for i in range(len(flattened)):
        normalized.append(flattened[i]/max(flattened))

    return normalized


def csv(letter, landmark_list):
    csv_path = 'keypoint.csv'
    with open(csv_path, 'a', newline="") as f:
        writer = csv.writer(f)
        writer.writerow([*landmark_list, letter])
    return

def landmark_list(image, landmarks):
    height, width = image.shape

    landmark_point = []

    for _, landmark in enumerate(landmarks.landmark):
        landmark_x = min(int(landmark.x * width), width - 1)
        landmark_y = min(int(landmark.y * height), height - 1)

        landmark_point.append([landmark_x, landmark_y])

    return landmark_point

In [49]:
import cv2
import numpy as np
import mediapipe as mp
import csv
import re
import os


def landmarks_to_csv():
  mp_drawing = mp.solutions.drawing_utils
  mp_drawing_styles = mp.solutions.drawing_styles
  mp_hands = mp.solutions.hands
  no_hand_count = 0

  pics = []

  DIR_SIZE = 500
  ITERATION = 1

  TRAIN_DIR = '../customdata/'

  for dir in os.listdir(TRAIN_DIR):
    ctr = 0
    for img in os.listdir(TRAIN_DIR + dir):
      if(ctr < DIR_SIZE):
        if ctr % ITERATION == 0:
          pics.append({TRAIN_DIR + dir + '/' + img:dir})
          # print(TRAIN_DIR + dir + '/' + img)
        ctr += 1

  with mp_hands.Hands(
    static_image_mode=True,
    max_num_hands=1,
    min_detection_confidence=0.5) as hands:
    
    print('starting the long part...')

    ctr = 0
    for file in pics:
      key = list(file.keys())[0]
      val = list(file.values())[0]
      if ctr % 1000 == 0:
        print('dir', val, 'img', key)
      ctr += 1
      image = cv2.flip(cv2.imread(key), 1)
      results = hands.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

      if not results.multi_hand_landmarks:
        no_hand_count += 1
        print('NO HANDS FOUND ', no_hand_count, key, val)
        continue

      for hand_landmarks in results.multi_hand_landmarks:
        landmark_list = landmark_list(image, hand_landmarks)
        pre_processed_list = pre_process_landmark(landmark_list)
        # print('logging', min(pre_processed_list))
        csv(val, pre_processed_list)
           

  print(no_hand_count, ' hands not found')  
  return


if __name__ == '__main__':
  landmarks_to_csv()


starting the long part...
dir A img ../customdata/A/A_0.jpg
dir C img ../customdata/C/C_0.jpg
dir E img ../customdata/E/E_0.jpg
dir G img ../customdata/G/G_0.jpg
dir I img ../customdata/I/I_0.jpg
dir K img ../customdata/K/K_0.jpg
dir M img ../customdata/M/M_0.jpg
dir O img ../customdata/O/O_0.jpg
dir Q img ../customdata/Q/Q_0.jpg
dir S img ../customdata/S/S_0.jpg
dir U img ../customdata/U/U_0.jpg
dir W img ../customdata/W/W_0.jpg
dir Y img ../customdata/Y/Y_0.jpg
dir [ img ../customdata/[/del_0.jpg
NO HANDS FOUND  1 ../customdata/]/space_156.jpg ]
NO HANDS FOUND  2 ../customdata/]/space_157.jpg ]
NO HANDS FOUND  3 ../customdata/]/space_159.jpg ]
NO HANDS FOUND  4 ../customdata/]/space_160.jpg ]
NO HANDS FOUND  5 ../customdata/]/space_161.jpg ]
NO HANDS FOUND  6 ../customdata/]/space_163.jpg ]
NO HANDS FOUND  7 ../customdata/]/space_164.jpg ]
NO HANDS FOUND  8 ../customdata/]/space_346.jpg ]
NO HANDS FOUND  9 ../customdata/]/space_349.jpg ]
NO HANDS FOUND  10 ../customdata/]/space_350.j

In [50]:
import csv
with open(r'keypoint.csv') as inputFile:
    i = 0
    x = csv.reader(inputFile)
    with open(r'new4.csv','w', newline='') as outFile:
        pass
    for lines in x:
        # print(lines)
            # print(len(lines))
            if len(lines) != 0 and len(lines[-1]) == 1:
                lines[-1] = np.float64(ord(lines[-1]) - 65 - int(lines[-1]==']'))
                with open(r'new4.csv','a', newline='') as outFile:
                    iWrite = csv.writer(outFile, delimiter=',')
                    iWrite.writerow(lines)

In [51]:
from sklearn.model_selection import train_test_split

RANDOM_SEED = 42

dataset = 'new4.csv'
model_save_path = 'keypoint_classifier2.hdf5'

test = X_dataset = np.loadtxt(dataset, delimiter=',')
X_dataset = np.loadtxt(dataset, delimiter=',', dtype='float32', usecols=list(range(0, (21 * 2))))
y_dataset = np.loadtxt(dataset, delimiter=',', dtype='int32', usecols=((21*2)))

X_train, X_test, y_train, y_test = train_test_split(X_dataset, y_dataset, train_size=0.75, random_state=RANDOM_SEED)

In [52]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Input((21 * 2, )),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(40, activation='relu'),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(30, activation='relu'),
    tf.keras.layers.Dense(28, activation='softmax')
])


cp_callback = tf.keras.callbacks.ModelCheckpoint(model_save_path, verbose=1, save_weights_only=False)
es_callback = tf.keras.callbacks.EarlyStopping(patience=20, verbose=1)

In [53]:
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [54]:
model.fit(
    X_train,
    y_train,
    epochs=500,
    batch_size=128,
    validation_data=(X_test, y_test),
    callbacks=[cp_callback, es_callback]
)

Epoch 1/500
84/88 [===========================>..] - ETA: 0s - loss: 3.2137 - accuracy: 0.1118
Epoch 00001: saving model to keypoint_classifier2.hdf5
88/88 [==============================] - 1s 9ms/step - loss: 3.2036 - accuracy: 0.1153 - val_loss: 2.8439 - val_accuracy: 0.2034
Epoch 2/500
88/88 [==============================] - ETA: 0s - loss: 2.6242 - accuracy: 0.2196
Epoch 00002: saving model to keypoint_classifier2.hdf5
88/88 [==============================] - 0s 5ms/step - loss: 2.6242 - accuracy: 0.2196 - val_loss: 1.9817 - val_accuracy: 0.3985
Epoch 3/500
76/88 [========================>.....] - ETA: 0s - loss: 2.0930 - accuracy: 0.3332
Epoch 00003: saving model to keypoint_classifier2.hdf5
88/88 [==============================] - 0s 5ms/step - loss: 2.0616 - accuracy: 0.3409 - val_loss: 1.4299 - val_accuracy: 0.6293
Epoch 4/500
85/88 [===========================>..] - ETA: 0s - loss: 1.7822 - accuracy: 0.4060
Epoch 00004: saving model to keypoint_classifier2.hdf5
88/88 [======

In [11]:
def getLandmarks(img):
    mp_drawing = mp.solutions.drawing_utils
    mp_drawing_styles = mp.solutions.drawing_styles
    mp_hands = mp.solutions.hands

    image = cv2.imread(img)


    with mp_hands.Hands(
        model_complexity=0,
        min_detection_confidence=0.5,
        min_tracking_confidence=0.5) as hands:

        results = hands.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

        if results.multi_hand_landmarks:
            print(results.multi_hand_landmarks)
            return results.multi_hand_landmarks[0]









In [27]:
# predicted_dataset = tf.convert_to_tensor([pre_processed_list])
# predictions = model(predicted_dataset, training=False)

# print(pre_processed_list)
# print(X_test[0])

# print(X_test[0])
prediction = model.predict(np.array([pre_processed_list]))
# prediction = model.predict(np.array([pre_processed_list]))
# print(np.squeeze(prediction))
print(np.argmax(np.squeeze(prediction)))
print(y_test[0])
# maxValue = max(prediction)
# print(np.argmax(prediction), prediction[0][np.argmax(prediction)])


# class_names = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W','X','Y','Z']
# for i, logits in enumerate(predictions):
#   class_idx = tf.argmax(logits).numpy()
#   p = tf.nn.softmax(logits)[class_idx]
#   name = class_names[class_idx]
#   print("Example {} prediction: {} ({:4.1f}%)".format(i, name, 100*p))

21
10


In [ ]:
train_dataset_url = "https://storage.googleapis.com/download.tensorflow.org/data/iris_training.csv"

train_dataset_fp = tf.keras.utils.get_file(fname=os.path.basename(train_dataset_url),
                                           origin=train_dataset_url)

print("Local copy of the dataset file: {}".format(train_dataset_fp))



Local copy of the dataset file: C:\Users\ricky\.keras\datasets\iris_training.csv


In [ ]:
import csv
with open(train_dataset_fp) as inputFile:
    i = 0
    x = csv.reader(inputFile)
    for lines in x:
        print(lines)
        

['120', '4', 'setosa', 'versicolor', 'virginica', ' a']
['6.4', '2.8', '5.6', '2.2', '2']
['5.0', '2.3', '3.3', '1.0', '1']
['4.9', '2.5', '4.5', '1.7', '2']
['4.9', '3.1', '1.5', '0.1', '0']
['5.7', '3.8', '1.7', '0.3', '0']
['4.4', '3.2', '1.3', '0.2', '0']
['5.4', '3.4', '1.5', '0.4', '0']
['6.9', '3.1', '5.1', '2.3', '2']
['6.7', '3.1', '4.4', '1.4', '1']
['5.1', '3.7', '1.5', '0.4', '0']
['5.2', '2.7', '3.9', '1.4', '1']
['6.9', '3.1', '4.9', '1.5', '1']
['5.8', '4.0', '1.2', '0.2', '0']
['5.4', '3.9', '1.7', '0.4', '0']
['7.7', '3.8', '6.7', '2.2', '2']
['6.3', '3.3', '4.7', '1.6', '1']
['6.8', '3.2', '5.9', '2.3', '2']
['7.6', '3.0', '6.6', '2.1', '2']
['6.4', '3.2', '5.3', '2.3', '2']
['5.7', '4.4', '1.5', '0.4', '0']
['6.7', '3.3', '5.7', '2.1', '2']
['6.4', '2.8', '5.6', '2.1', '2']
['5.4', '3.9', '1.3', '0.4', '0']
['6.1', '2.6', '5.6', '1.4', '2']
['7.2', '3.0', '5.8', '1.6', '2']
['5.2', '3.5', '1.5', '0.2', '0']
['5.8', '2.6', '4.0', '1.2', '1']
['5.9', '3.0', '5.1', '1.8

In [ ]:
# column order in CSV file
column_names = ['sepal_length', 'sepal_width', 'petal_length', 'petal_width', 'species']

feature_names = column_names[:-1]
label_name = column_names[-1]

print("Features: {}".format(feature_names))
print("Label: {}".format(label_name))

batch_size = 32

train_dataset = tf.data.experimental.make_csv_dataset(
    train_dataset_fp,
    batch_size,
    column_names=column_names,
    label_name=label_name,
    num_epochs=1)

features, labels = next(iter(train_dataset))

print(features)

import csv
with open(train_dataset_fp) as inputFile:
    i = 0
    x = csv.reader(inputFile)
    for lines in x:
        print(lines)


Features: ['sepal_length', 'sepal_width', 'petal_length', 'petal_width']
Label: species


ValueError: Problem inferring types: CSV row 2 has 6 number of fields. Expected: 5.